In [1]:
wb = spark.read.json('/FileStore/tables/world_bank.json')

In [2]:
wb.show(2)

In [3]:
wb.printSchema()

In [4]:
wb.createOrReplaceTempView('world_bank')

In [5]:
df1 = spark.sql("select countryshortname from world_bank")
df1.show(2)

In [6]:
wb.createGlobalTempView("bhaskar")

In [7]:
df3 = spark.sql("SELECT  distinct approvalfy FROM global_temp.bhaskar WHERE countryshortname = 'Tunisia' ")

In [8]:
df3.show()

In [9]:
df4 = spark.sql("SELECT  distinct countryshortname FROM global_temp.bhaskar WHERE approvalfy = '2013' ")

In [10]:
df4.count()

In [11]:
df3 = spark.sql("SELECT  distinct approvalfy FROM global_temp.bhaskar WHERE countryshortname = 'Tunisia' ")
df4 = spark.sql("SELECT  distinct countryshortname FROM global_temp.bhaskar WHERE approvalfy = '2013' ")
print df3.show()
print df.count()

In [12]:
zpis = spark.read.json('/FileStore/tables/zips.json')
stocks = spark.read.json('/FileStore/tables/stocks.json')
enron = spark.read.json('/FileStore/tables/enron.json')
companies = spark.read.json('/FileStore/tables/companies.json')

In [13]:
enron.printSchema()

In [14]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

def getter(column):

    for i,col in enumerate(column):
        if i==1:
           col_new=col
        else:
           col_new=col_new+','+col
    return col_new

getterUDF = udf(getter, StringType())

df.select(getterUDF(enron.cc))

In [15]:
#zpis[['loc']].show()
from pyspark.sql.functions import to_json, from_json, col, struct, lit
from pyspark.sql.types import StructType, StructField
from pyspark.ml.linalg import VectorUDT

json_vec = to_json(struct(struct(
    lit(1).alias("type"),  # type 1 is dense, type 0 is sparse
    col("zpis.loc").alias("values")
).alias("v")))

schema = StructType([StructField("v", VectorUDT())])

with_parsed_vector = df.withColumn(
    "parsed_vector", from_json(json_vec, schema).getItem("v")
)

with_parsed_vector.show()

In [16]:
zpis.write.parquet("zpis1.parquet")

In [17]:
zpis1 = spark.read.parquet('zpis1.parquet')

In [18]:
print zpis1.show(2)
print zpis.show(2)

In [19]:
#zpis1.createGlobalTempView('zpis1')

spark.catalog.refreshTable("global_temp.zpis1")

In [20]:
zpis.printSchema()

In [21]:
import pyspark.ml.feature as ft

In [22]:
transformed_id = ft.StringIndexer(inputCol='_id', outputCol='id_en')
transformed_city = ft.StringIndexer(inputCol='city', outputCol='city_en')
transformed_state = ft.StringIndexer(inputCol='state', outputCol='state_en')

In [23]:
transfor

In [24]:
featurescreator = ft.VectorAssembler(inputCols=['id_en',
                                     'city_en',
                                     "features[{2}]". format(i) for i in range(n)
                                     'state_en' ], outputCol='features')

In [25]:
import pyspark.ml.classification as cl

In [26]:
createLogistic = cl.LogisticRegression(maxIter=10, regParam=0.01, labelCol='pop')

In [27]:
from pyspark.ml import Pipeline

In [28]:
pipeline = Pipeline(stages=[transformed_id,
                            transformed_city,
                            transformed_state,
                           featurescreator,
                           createLogistic])

In [29]:
zpis_train, zpis_test = zpis.randomSplit([0.7, 0.3], seed=100)

In [30]:
model = pipeline.fit(zpis_train)

In [31]:
test_out = model.transform(zpis_test)

In [32]:
test_out

In [33]:
#To find accuracy of the algo under processing
import pyspark.ml.evaluation as ev

In [34]:

#BinaryClassification
evaluator = ev.BinaryClassificationEvaluator(rawPredictionCol='probability', 
                                             labelCol='pop')

In [35]:
evaluator.evaluate(test_out)

In [36]:
zpis = spark.read.json('/FileStore/tables/zips.json')

In [37]:
from pyspark.sql.functions import to_json, from_json, col, struct, lit
from pyspark.sql.types import StructType, StructField
from pyspark.ml.linalg import VectorUDT

json_vec = to_json(struct(struct(lit(1).
                                  col('loc')
).alias("v")))

schema = StructType([StructField("v", VectorUDT())])

with_parsed_vector = df.withColumn(
    "parsed_vector", from_json(json_vec, schema).getItem("v")
)

In [38]:
with_vec = df.withColumn("loc", as_vector("loc"))
with_vec.show()

In [39]:
zpis.show(2)

In [40]:
companies = spark.read.json('/FileStore/tables/companies.json')

In [41]:
companies.show(2)